<a href="https://colab.research.google.com/github/adheeb4/7PAM2015-0105-2023-Individual-Project/blob/main/Individual_Project_Image_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install fiftyone
!pip install fiftyone-db==0.4.3

In [17]:
import fiftyone as fo
import fiftyone.zoo as foz
import numpy as np
import cv2

In [ ]:
dataset = foz.load_zoo_dataset("coco-2017",
splits=["train", "validation", "test"],
#classes=["person", "car", "dog", "cake"]
                               )

INFO:fiftyone.zoo.datasets:Downloading split 'train' to '/root/fiftyone/coco-2017/train' if necessary


Found annotations at '/root/fiftyone/coco-2017/raw/instances_train2017.json'


INFO:fiftyone.utils.coco:Found annotations at '/root/fiftyone/coco-2017/raw/instances_train2017.json'


Found 9 (< 118287) downloaded images; must download full image zip


INFO:fiftyone.utils.coco:Found 9 (< 118287) downloaded images; must download full image zip


INFO:fiftyone.utils.coco:Downloading images to '/root/fiftyone/coco-2017/tmp-download/train2017.zip'


 100% |████|  144.1Gb/144.1Gb [5.7m elapsed, 0s remaining, 389.1Mb/s]       


INFO:eta.core.utils: 100% |████|  144.1Gb/144.1Gb [5.7m elapsed, 0s remaining, 389.1Mb/s]       


Extracting images to '/root/fiftyone/coco-2017/train/data'


INFO:fiftyone.utils.coco:Extracting images to '/root/fiftyone/coco-2017/train/data'


In [25]:
print(dataset)

Name:        coco-2017-train-validation-test-5
Media type:  image
Num samples: 15
Persistent:  False
Tags:        []
Sample fields:
    id:           fiftyone.core.fields.ObjectIdField
    filepath:     fiftyone.core.fields.StringField
    tags:         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    ground_truth: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)


In [26]:
num_images = len(dataset)
print(f"Number of images: {num_images}")

Number of images: 15


In [31]:
#print(dataset.fields)

# Convert the dataset to a list for iteration
samples = list(dataset)

# Access the first sample and print its field names
sample = samples[0]
print(sample.keys())

AttributeError: Sample has no field 'keys'

In [32]:
# Load the COCO-2017 dataset from the FiftyOne Zoo
#dataset = foz.load_zoo_dataset("coco-2017", split="train", dataset_name="coco-2017-train-sample")

# Access images and their dimensions
images = list(dataset)  # Get a list of all samples in the dataset
image_sizes = [(img["image"].shape[0], img["image"].shape[1]) for img in images]

# Plot image size distribution
plt.figure(figsize=(8, 6))
plt.hist([w for (w, h) in image_sizes], bins=20, label="Width")
plt.hist([h for (w, h) in image_sizes], bins=20, alpha=0.7, label="Height")
plt.legend()
plt.xlabel("Image Dimension")
plt.ylabel("Frequency")
plt.title("Image Size Distribution")
plt.show()

KeyError: "Sample has no field 'image'"

In [33]:
session = fo.launch_app(dataset)

In [ ]:


def preprocess(sample, resize_shape=(256, 256)):
  image_data = cv2.imread(sample.filepath)
  binary_mask = np.zeros(image_data.shape[:2], dtype=np.uint8)

  detections = sample["detections"]
  if detections is not None:
    for detection in detections.values():
      for seg in detection.segmentation:
        cv2.fillPoly(binary_mask, [np.array(seg)], 255)
  #for seg in sample.mask.ground_truth.segmentation:
   # cv2.fillPoly(binary_mask, np.array(seg), 255)
  if resize_shape:
    image_data = cv2.resize(image_data, resize_shape)
    binary_mask = cv2.resize(binary_mask, resize_shape, interpolation=cv2.INTER_NEAREST)

  image_data = image_data / 255.0

  return image_data, binary_mask


preprocessed_images = []
preprocessed_masks = []

for sample in dataset:
 # image = sample["image"]
#mask = sample["ground_truth"]

  preprocessed_image, preprocessed_mask = preprocess(sample)

  preprocessed_images.append(preprocessed_image)
  preprocessed_masks.append(preprocessed_mask)


KeyError: "Sample has no field 'detections'"

In [ ]:
view.update(fields="image", value=preprocessed_image)

NameError: name 'view' is not defined